# Speed measure def

In [1]:
SPLIT_TRAIN_SESSIONS = {}
SPLIT_TRAIN_SESSIONS["A"] = {"session0"}
SPLIT_TRAIN_SESSIONS["B"] = SPLIT_TRAIN_SESSIONS["A"] | {"session1", "session2"}
a = [1,2,3]
print(set(a))

set([1, 2, 3])


In [2]:
# DATASET_BASE_PATH = TRANS_PATH("../dataset/")
# RESULTS_DIR = TRANS_PATH("../results/")

DATASET_SESSIONS = {
    "session0": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 25.0},
            "right": {"fps": 50.0}
        }
        
    },
    "session1": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        }
    },
    "session2": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        }
    },
    "session3": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        }
    },
    "session4": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        }
    },
    "session5": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        }
    },
    "session6": {
        "recordings": {        
            "left": {"fps": 50.0},
            "center": {"fps": 50.0},
            "right": {"fps": 50.0}
        },
        "invalidLanes": set([2])
        
    }
    
}





ALL_SESSIONS = set(DATASET_SESSIONS.keys())

ALL_VIDEOS = []
for _sId in sorted(DATASET_SESSIONS):
    for _rId in ("left", "center", "right"):
        ALL_VIDEOS.append((_sId, _rId))

SPLIT_TRAIN_SESSIONS = {}
SPLIT_TRAIN_SESSIONS["A"] = {"session0"}
SPLIT_TRAIN_SESSIONS["B"] = SPLIT_TRAIN_SESSIONS["A"] | {"session1", "session2"}
SPLIT_TRAIN_SESSIONS["C"] = SPLIT_TRAIN_SESSIONS["B"] | {"session3"}

SPLIT_TEST_SESSIONS = dict(map(lambda i: (i[0], ALL_SESSIONS-i[1]), SPLIT_TRAIN_SESSIONS.iteritems()))

SPLIT_TRAIN_VIDEOS = dict(map(lambda i: (i[0], filter(lambda j: j[0] in i[1], ALL_VIDEOS)), SPLIT_TRAIN_SESSIONS.iteritems()))
SPLIT_TEST_VIDEOS = dict(map(lambda i: (i[0], filter(lambda j: j[0] in i[1], ALL_VIDEOS)), SPLIT_TEST_SESSIONS.iteritems()))

# print SPLIT_TRAIN_SESSIONS
# print ""
# print SPLIT_TEST_SESSIONS
# print ""
# print SPLIT_TRAIN_VIDEOS
# print ""
# print SPLIT_TEST_VIDEOS

In [26]:
import numpy as np
def isPointBetweenLines(p, l1, l2):
    return np.dot(p,l1)*np.dot(p,l2)*np.dot(l1[0:2],l2[0:2]) <= 0
def getLaneForPoint(p, lines):
    for i in xrange(len(lines)-1):
        if isPointBetweenLines(p, lines[i], lines[i+1]):
            return i
    return None
p = np.array([1,2,1])
l1 = np.array([1,2,3])
l2 = np.array([2,2,3])
l3 = np.array([3,2,3])
l = [l1,l2,l3]
print(getLaneForPoint(p,l))

None


In [4]:
from __future__ import division
from __future__ import print_function
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import cPickle
def pointToLineDistance(p, l):
    return abs(np.dot(l,p/p[2]))/np.linalg.norm(l[0:2])

def pointToLineProjection(l, p):
    p = p/p[-1]
    c = p[0]*l[1] - p[1]*l[0]
    perpendicularLine = np.array([-l[1], l[0], c])
    intersection = np.cross(l, perpendicularLine)
    return intersection/intersection[-1]


In [5]:
from __future__ import division
from __future__ import print_function
import numpy as np
import math

def getFocal(vp1, vp2, pp):
    return math.sqrt(- np.dot(vp1[0:2]-pp[0:2], vp2[0:2]-pp[0:2]))

def isPointBetweenLines(p, l1, l2):
    return np.dot(p,l1)*np.dot(p,l2)*np.dot(l1[0:2],l2[0:2]) <= 0

def computeCameraCalibration(_vp1, _vp2, _pp):
    vp1 = np.concatenate((_vp1, [1]))    
    vp2 = np.concatenate((_vp2, [1]))    
    pp = np.concatenate((_pp, [1]))    
    focal = getFocal(vp1, vp2, pp)
    vp1W = np.concatenate((_vp1, [focal]))    
    vp2W = np.concatenate((_vp2, [focal]))    
    ppW = np.concatenate((_pp, [0])) 
    vp3W = np.cross(vp1W-ppW, vp2W-ppW)
    vp3 = np.concatenate((vp3W[0:2]/vp3W[2]*focal + ppW[0:2], [1]))
    vp3Direction = np.concatenate((vp3[0:2], [focal]))-ppW
    roadPlane = np.concatenate((vp3Direction/np.linalg.norm(vp3Direction), [10]))
    return vp1, vp2, vp3, pp, roadPlane, focal

def getWorldCoordinagesOnRoadPlane(p, focal, roadPlane, pp):
    p = p/p[2]
    pp = pp/pp[2]
    ppW = np.concatenate((pp[0:2], [0]))
    pW = np.concatenate((p[0:2], [focal]))
    dirVec = pW - ppW
    t = -np.dot(roadPlane, np.concatenate((ppW, [1])))/np.dot(roadPlane[0:3], dirVec)
    return ppW + t*dirVec

vp1 = np.array([144.737, 34.7794])
vp2 = np.array([12183.582175112755, 615.451021479187])
pp = np.array([960, 540])
computeCameraCalibration(vp1,vp2,pp)


(array([144.737 ,  34.7794,   1.    ]),
 array([1.21835822e+04, 6.15451021e+02, 1.00000000e+00]),
 array([8.76075254e+00, 2.02616837e+04, 1.00000000e+00]),
 array([960, 540,   1]),
 array([-0.04761926,  0.98727215,  0.15174356, 10.        ]),
 3031.219339667006)

In [29]:
from __future__ import division

import matplotlib
matplotlib.rcParams['text.latex.unicode'] = True
font = {'size'   : 13}
matplotlib.rc('font', **font)

import scipy.ndimage
import scipy.stats
import scipy.optimize
import matplotlib.pyplot as plt
import json
import scipy.stats
from tabulate import tabulate
import operator
import itertools
import sys
import argparse
from matplotlib.ticker import MultipleLocator

def getPathForRecording(session, recording):
    return os.path.join('./dataset/', "%s_%s"%(session, recording))

def getCarTimeAndSpatialIntersection(line, posX, posY, frames, around=6):
    points = map(lambda x,y,f: (np.array([x,y, 1]),f), posX, posY, frames)
    points.sort(key=lambda i: pointToLineDistance(i[0], line))
    getItems = lambda getter: map(getter, points[:min(around,len(points))])
    spatialLineSlope, spatialLineIntercept = scipy.stats.linregress(getItems(lambda i: i[0][0]),
                                                                    getItems(lambda i: i[0][1]))[:2]
    spatialLine = np.array([spatialLineSlope, -1, spatialLineIntercept])
    spatialIntersection = np.cross(spatialLine, line)
    spatialIntersection = spatialIntersection/spatialIntersection[-1]

    frameNumbers = getItems(lambda i: i[1])
    normalizationPoint = np.cross(spatialLine, [0,1,0])
    normalizationPoint = normalizationPoint/normalizationPoint[-1]
    pointsProjected = map(lambda p: pointToLineProjection(spatialLine, p[0]), points[:min(around,len(points))])
    pointsDists = map(lambda p: np.linalg.norm(normalizationPoint-p), pointsProjected)
    temporalLineSlope, temporalLineIntercept = scipy.stats.linregress(pointsDists,frameNumbers)[:2]
    temporalIntersection = temporalLineSlope * np.linalg.norm(spatialIntersection-normalizationPoint) + temporalLineIntercept

    return spatialIntersection, temporalIntersection, spatialLine


def pointToLineProjection(l, p):
    p = p/p[-1]
    c = p[0]*l[1] - p[1]*l[0]
    perpendicularLine = np.array([-l[1], l[0], c])
    intersection = np.cross(l, perpendicularLine)
    return intersection/intersection[-1]
        
def pointToLineDistance(p, l):
    return abs(np.dot(l,p/p[2]))/np.linalg.norm(l[0:2])

def calculateSpeeds(data, gtData, us = False, show = ['frame',1]):
    '''
    Okay, now we will find the meaning of each line of code. I'm sure this guy is a student. A clever student
    '''    
    # this line, of course, no need to explain
    if us == False:
        vp1, vp2, vp3, pp, roadPlane, focal = computeCameraCalibration(data["camera_calibration"]["vp1"],
                                                            data["camera_calibration"]["vp2"],
                                                            data["camera_calibration"]["pp"])
    elif us == True:
        vp1 = np.array([144.737, 34.7794])
        vp2 = np.array([12183.5821511755, 615.451021479187])
        pp = np.array([960,540])
#         vp1 = np.array([133.5099, 31.3907])
#         vp2 = np.array([10794,774.1463])
        vp1, vp2, vp3, pp, roadPlane, focal = computeCameraCalibration(vp1,
                                                                vp2,
                                                                pp)
    #     vp3 = np.array([945.0162,1002,1])
#         focal = 461.3994
#         roadPlane = np.array([-0.0229, 0.7074, 0.7064, 10])

    # ah, lambda, remember it, my friend? 1 way in, 1 value out, impress. This line take the path of gtData
#     pTran = lambda p: os.path.join(getPathForRecording(sessionId, recordingId), p)
    lines = gtData["measurementLines"]
    laneDivLines = gtData["laneDivLines"]
    laneDivLines[3]=np.array([0.57,-1,-47.39])
    fps = gtData["fps"]
    errors = 0
    count_car = 0

    for car in data["cars"]:
        if car['id'] not in temp:
            continue
        intersectionData = map(lambda l: getCarTimeAndSpatialIntersection(l, car["posX"], car["posY"], car["frames"]), lines)
        startSpatial, startFrame, startLineSpatial = intersectionData[-1]
        endSpatial, endFrame, endLineSpatial = intersectionData[0]
#         test on poorly approximated intersections
#         testMat = np.concatenate((endSpatial, startSpatial,
#                                   [startFrame, endFrame]))
        startPoint = [car["posY"][0],car["posX"][0],1]
        endPoint = [car["posY"][-1],car["posX"][-1],1]
        '''
        debug, fourth lane
        '''
#         car_with_problem = [279,297,422]
#         if car['id'] in car_with_problem:
#             print(car['id'])
#             print(np.any(np.isnan(testMat)))
#             print(np.any(np.isinf(testMat)))
#             print(getLaneForPoint(endSpatial, laneDivLines))
#             print(getLaneForPoint(startSpatial, laneDivLines))
#             print(testMat)
#             print((np.any(np.isnan(testMat)) or \
#                 np.any(np.isinf(testMat)) or \
#                 getLaneForPoint(endSpatial, laneDivLines) is None or \
#                 getLaneForPoint(startSpatial, laneDivLines) is None))
#             print(startSpatial,endSpatial)
#             print(startPoint,endPoint)
#             print(getLaneForPoint(endPoint, laneDivLines))
#             print(getLaneForPoint(startPoint, laneDivLines))
#             print(laneDivLines)
       
        
#         if not (np.any(np.isnan(testMat)) or \
#                 np.any(np.isinf(testMat)) or \
#                 getLaneForPoint(endSpatial, laneDivLines) is None or \
#                 getLaneForPoint(startSpatial, laneDivLines) is None):
        
        
        '''
        debug, close bug
        '''
        
        if not(getLaneForPoint(startPoint, laneDivLines) is None or \
               getLaneForPoint(endPoint, laneDivLines) is None):
            if count_car == show[1]:
                break
#             elapsedTime = abs(endFrame-startFrame)/fps
            projector = lambda p: getWorldCoordinagesOnRoadPlane(p, focal, roadPlane, pp)
#             passedDistance = data["camera_calibration"]["scale"]*np.linalg.norm(projector(startSpatial)-projector(endSpatial))
#             car["speed"] = passedDistance/elapsedTime * 3.6
#             print('Best measure: {}'.format(car["speed"]))
#             car["laneIndex"] = getLaneForPoint(endSpatial, laneDivLines)
#             car["timeIntersectionLast"] = endFrame/fps

#             car["timeIntersectionFirst"] = startFrame/fps
#             car["spatialIntersectionFirst"] = startSpatial
#             car["spatialIntersectionLast"] = endSpatial
            points = map(lambda x,y: np.array([x,y,1]), car["posX"], car["posY"])
            points = map(projector, points)
            frames = car["frames"]
            perFrameSpeeds = []
            lanes = []
            pointsOffset = 5
            '''
            Toby
            '''
            count = 0
            for i in xrange(len(points)-pointsOffset):
                passedDistance = data["camera_calibration"]["scale"]*np.linalg.norm(points[i]-points[i+pointsOffset])
                elapsedTime = abs(frames[i]-frames[i+pointsOffset])/fps
                p = np.array([car['posY'][count],car['posX'][count],1])
                '''
                Toby
                '''
#                 if show[0] == 'frame':
#                     print('At frame: {} with ID: {} is {}'.format(car['frames'][count],car['id'],passedDistance/elapsedTime * 3.6))
                perFrameSpeeds.append(passedDistance/elapsedTime * 3.6)
                lanes.append(getLaneForPoint(p,laneDivLines))
                count+=1
            
            car['speedOffset'] = perFrameSpeeds
            car["medianSpeed"] = np.median(perFrameSpeeds)
            car['lanes'] = lanes
#             if show[0] == 'median' and count_car < show[1]:
#                 print(str(car['id']) + ': ' + str(car["medianSpeed"]))
            count_car+=1
    return data

def showSpeeds(video_path, data, output):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    if not output is None:
        out = cv2.VideoWriter(output,cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width,frame_height))
    window_name = 'frame'
    cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 3
    frame_num = 0
    if (cap.isOpened()==False):
        print('Please check the video_path!!!')   
    while(cap.isOpened()):
        ret,frame = cap.read()
        if frame_num in data.keys():
            fault = True
            frame_fault = 0
            text = 'Normal'
            for package in data[frame_num]:
                if package[5]==1:
                    text = 'Fault'
                frame = cv2.putText(frame, 'ID: '+str(int(package[2])), (int(package[1]-10),int(package[0]-10)), font,  
                       fontScale, color, thickness, cv2.LINE_AA) 
                frame = cv2.putText(frame, 'Speed: '+str(package[3]), (int(package[1])+10,int(package[0])+10), font,  
                       fontScale, color, thickness, cv2.LINE_AA)
                frame = cv2.putText(frame, 'Lane: '+str(package[4]), (int(package[1])+30,int(package[0])+30), font,  
                       fontScale, color, thickness, cv2.LINE_AA)
                frame = cv2.putText(frame, 'Cross the line: '+ text, (int(package[1])+50,int(package[0])+50), font,  
                       fontScale, color, thickness, cv2.LINE_AA)
        if not output is None:
            out.write(frame)
        cv2.resizeWindow(window_name, 900,600)
        cv2.imshow(window_name, frame)
        frame_num+=1
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    if not output is None:
        out.release()
    cv2.destroyAllWindows()

In [7]:
import numpy as np
def findCrossLines(lanes):
    #lanes is a list contain each lane of 1 ID in each frame
    check = [0]
    fault = 0
    for i in xrange(len(lanes)-1):   
        if lanes[i] != lanes[i+1]:
            fault = 1
        check.append(fault)
    return check
        
def sortByFrame(data):
    first = True
    for car in data['cars']:
        try:
            lenSpeed = len(car['speedOffset'])
        except:
            continue
        posX = np.array(car['posX'][:lenSpeed], ndmin = 2)
        posY = np.array(car['posY'][:lenSpeed], ndmin = 2)
        frame = np.array(car['frames'][:lenSpeed], ndmin = 2)
        ID = np.full((1,lenSpeed), car['id'])
        speed = np.array(car['speedOffset'], ndmin = 2)
        lane = np.array(car['lanes'], ndmin = 2)
        check = map(lambda l: findCrossLines(l), lane)
        package = np.concatenate((posX,posY,ID,speed,lane,check,frame), axis = 0).T
        if first:
            all_package = package
            first = False
        else:
            all_package = np.concatenate((all_package, package))
#     all_package = np.asarray(sorted(all_package, key=lambda x: x[4]))
    all_package = all_package[all_package[:,6].argsort()]
    dict_package = {}
    for package in all_package:
        if package[6] in dict_package.keys():
            dict_package[package[6]].append(package[:6])
        else:
            dict_package[package[6]] = [package[:6]]
    return dict_package



# Session0_center

In [11]:
import pickle
import cPickle
from matplotlib import pyplot as plt
# for lap
# data_link = "C:/Users/TobyCurtis/Desktop/final_project/BrnoCompSpeed/code/dataset/session0_center/gt_data.pkl"

# for PC
data_link = "C:/Users/ADMINS/Desktop/final_project/BrnoCompSpeed/code/dataset/session0_center/gt_data.pkl"

with open(data_link, 'rb') as fid:
    gtData = cPickle.load(fid)

In [31]:
for key, value in gtData.iteritems() :
    print(key)
print(gtData['laneDivLines'])

distanceMeasurement
measurementLines
cars
fps
invalidLanes
laneDivLines
[array([ -0.92226493,   0.38655841, 130.89136422]), array([-0.78325241,  0.62170384, 97.52101372]), array([-0.65281474,  0.7575176 , 71.40300615]), array([  0.57,  -1.  , -47.39])]


In [36]:
a = set()
a.add(1)
a.add(2)
print(3 in a)

False


In [30]:
from __future__ import division
from __future__ import print_function
import json

gD = gtData
system = 'GPsys'

# video path on lap
video = 'C:/Users/TobyCurtis/Desktop/front.avi'

#video path on PC
# video = 'Z:/Vehicle speed measuring/video_speed measure/front.avi'

# Brno data on lap
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/BrnoCompSpeed/code/results/session0_center/system_dubska_optimal_calib.json'))

# Brno data on PC
# data = json.load(open('C:/Users/ADMINS/Desktop/final_project/BrnoCompSpeed/code/results/session0_center/system_dubska_optimal_calib.json'))

# Our data on lap
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/result.json'))

# Our data on PC
# data = json.load(open('C:/Users/ADMINS/Desktop/final_project/result.json'))

# Our data from middle on lap
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/result_500.json'))

# Our data from middle from PC
# data = json.load(open('C:/Users/ADMINS/Desktop/final_project/result_500.json'))

# Brand new
data = json.load(open('C:/Users/ADMINS//Desktop/final_project/result_new.json'))
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/result_new.json'))

data = calculateSpeeds(data, gD, us = True, show = ['median',50])
# data_show = sortByFrame(data)
# showSpeeds(video, data_show,'C:/Users/ADMINS/Desktop/speedAndLines.avi')
# showSpeeds(video, data_show,'C:/Users/TobyCurtis/Desktop/speedAndLines.avi')


285
[23.324475700058198, 25.925994571897558, 28.298018793740148, 30.989956522046185, 29.72617349357867, 29.366411999684345, 30.25140696519203, 28.994871561195765, 28.41367849744269, 29.309160188222506, 29.264883013050273, 28.57252719422328, 29.61737991881427, 29.98325665124152, 29.308597719920233, 29.25071808418721, 28.56823469346728, 28.718231064818713, 28.240129908829555, 27.846183251649666, 28.181537274230028, 29.035610041738195, 28.273519343960174, 27.983564137728262, 28.968923477026173, 28.344797983498054, 28.04521117701355, 28.904687489952735, 29.392496472738642, 28.846171621761304, 29.32886908133587, 28.746763402159132, 27.27580786740102, 27.94088273817173, 27.046991678732955, 26.819420715009393, 26.245523554803714, 27.864064745703125, 25.584927373456775, 27.533813751848907, 27.13158054232966, 26.896139834900833, 27.85450432234008, 28.327149411043187]
301
[17.78510707982839, 20.018349241169, 22.849490150097374, 23.657839543550985, 23.022359638536198, 24.082292459881682, 23.06183

In [115]:
# 287 285, opposite

10

In [60]:
def isPointBetweenLines(p, l1, l2):
    return np.dot(p,l1)*np.dot(p,l2)*np.dot(l1[0:2],l2[0:2]) <= 0
l1 = np.array([-0.65281474,  0.7575176 , 71.40300615])
l2 = np.array([  0.57,  -1.  , -47.39])
p = np.array([499.26340711, 635.35478291,   1.        ])
isPointBetweenLines(p,l1,l2)


False

In [51]:
def searchCarByID(data, ID):
    for car in data['cars']:
        if car['id'] == ID:
            return car
b = searchCarByID(data, 297)
print(b['posX'][0],b['posY'][0])
# print(b['posX'][0],b['posX'][-1])   

301.268432664 544.844186231


In [32]:
testMat = [526.35917247, 871.71218078,   1. ,        366.15914674, 634.74802283
   ,1.         ,190.08498706, 221.11295049]
np.any(np.isnan(testMat))
# print(np.any(np.isnan(testMat)))
# print(np.any(np.isinf(testMat)))


False

In [21]:
from __future__ import division
from __future__ import print_function
import json

gD = gtData
system = 'GPsys'
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/BrnoCompSpeed/code/results/session0_center/system_dubska_optimal_calib.json'))
# data = json.load(open('C:/Users/ADMINS/Desktop/result.json'))
data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/result_500.json'))

temp = calculateSpeeds(data, gD, us = True, show = ['median',10])

IOError: [Errno 2] No such file or directory: 'C:/Users/TobyCurtis/Desktop/final_project/result_500.json'

In [37]:
from __future__ import division
from __future__ import print_function
import json

gD = gtData
system = 'GPsys'
# data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/BrnoCompSpeed/code/results/session0_center/system_dubska_optimal_calib.json'))
# data = json.load(open('C:/Users/ADMINS/Desktop/result.json'))
data = json.load(open('C:/Users/TobyCurtis/Desktop/final_project/result.json'))

temp = calculateSpeeds(data, gD, us = False, show = ['median',10])

281: 45.67227951968226
280: 51.68199025219607
279: 50.15425686516224
300: nan
321: nan
328: nan
350: 50.7190528938274
352: nan
369: nan
398: 25.75439883764166


In [38]:
for k,i in enumerate(data['cars']):
    if i['id']==278:
        print(k)
        print(i['frames'])

3
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 1

In [39]:
print(data['cars'][3])

{u'frames': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0

In [33]:
print('Ground truth: {}'.format(gD['cars'][0]['speed']))
print('Optimal_calib_system: {}'.format(temp['cars'][0]['speed']))

Ground truth: 75.9129425052


KeyError: 'speed'

# Session0-left